In [1]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [2]:
!nvidia-smi

Thu Oct  2 10:27:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Download Dataset
Selanjutnya, unduh dataset fashion_mnist. Dataset ini sudah tersedia pada TensorFlow sehingga kita bisa mengunduh dataset tersebut dengan mudah dengan perintah berikut:

In [3]:
fashion = datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Untuk mendapatkan performa yang baik, lakukan normalisasi nilai piksel gambar sehingga bernilai di antara 0 dan 1 (0 – 1):



In [4]:
X_train = X_train/255.0
X_test  = X_test/255.0

Siapkan model NN sesuai dengan arsitektur yang telah ditetapkan. Pada eksperimen ini, kita menggunakan NN dengan satu hidden layer yang terdiri dari 32 node dan output layer yang terdiri dari 10 node:



In [5]:
model = Sequential()
model.add(Flatten(input_shape=[28, 28]))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Terdapat dua cara untuk membuat model jaringan pada TensorFlow dan Keras: menggunakan Sequential Model dan Functional API. Eksperimen ini menggunakan Sequential Model dengan membuat model secara bertahap dengan fungsi add.

Perhatikan, ukuran input layer disesuaikan dengan dimensi gambar fashion_mnist, yaitu 28×28 dan fungsi aktivasi pada hidden layer pertama digunakan nilai default, yaitu ReLu.

Pada masalah klasifikasi dengan setiap example hanya memiliki satu kategori atau kelas (seperti pada kasus klasifikasi gambar pakaian ini) maka setiap neuron pada output layer mewakili satu kategori dengan fungsi aktivasi Softmax. Fungsi aktivasi Softmax pada output layer akan memastikan masing-masing nilai kemungkinan pada neuron-neuron tersebut berada di antara 0 dan 1 serta total nilai-nilai kemungkinan tersebut pada saat dijumlahkan adalah 1.

Lakukan kompilasi model yang telah didefinisikan di atas dengan menggunakan optimizer SGD:



In [6]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

sparse_categorical_crossentropy digunakan sebagai loss function karena studi kasus ini merupakan kategorisasi fashion dengan label berbentuk angka/integer 0 – 9. Tetapi, jika label telah dilakukan pengkodean dengan one-hot encoding maka loss function categorical_crossentropy yang digunakan.

Studi kasus ini merupakan Supervised Learning dengan masalah klasifikasi. Karena terdapat 10 kategori pada data label maka ini merupakan Multiclass Classification.

Mari kita lihat struktur jaringan yang telah dikompilasi:

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,450 (99.41 KB)

 Trainable params: 25,450 (99.41 KB)

 Non-trainable params: 0 (0.00 B)

Perhatikan, jumlah neuron pada input layer sebesar 784 yang merupakan hasil flatten dari gambar input berukuran 28×28. Input layer tidak memiliki paramater jaringan sehingga bernilai 0. Sedangkan pada hidden dan output layer masing-masing terdapat 25.120 dan 330 paramater jaringan sehingga eksperimen ini akan melatih total 25.450 paramater. None pada Output Shape merupakan ukuran batch yang akan ditetapkan atau dihitung kemudian pada saat pelatihan.

Lakukan pelatihan model sebanyak 50 epochs dengan batch_size = 1 untuk eksperimen SGD_1, batch_size = 32 untuk eksperimen SGD_32 dan batch_size = 60000 untuk eksperimen SGD_60k. Perintah di bawah ini adalah untuk SGD_32:

In [8]:
model.fit(X_train, y_train, epochs=50, validation_split=0.2, batch_size=32)

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6176 - loss: 1.2055 - val_accuracy: 0.7940 - val_loss: 0.5981
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8093 - loss: 0.5786 - val_accuracy: 0.8192 - val_loss: 0.5203
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8267 - loss: 0.5124 - val_accuracy: 0.8276 - val_loss: 0.4881
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8334 - loss: 0.4819 - val_accuracy: 0.8342 - val_loss: 0.4682
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8444 - loss: 0.4535 - val_accuracy: 0.8392 - val_loss: 0.4543
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8459 - loss: 0.4458 - val_accuracy: 0.8443 - val_loss: 0.4420
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8483 - loss: 0.4385 - val_accuracy: 0.8426 - val_loss: 0.4445
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8511 - loss: 0.4260 - 

Loss dan accuracy pada output di atas adalah nilai error dan akurasi yang telah dihitung dengan Training Set, sedangkan val_loss dan val_accuracy merupakan nilai error dan akurasi yang telah dihitung dengan Validation Set. Tentu yang menjadi perhatian kita di sini adalah val_loss dan val_accuracy.

Terakhir, lakukan evaluasi terhadap model yang telah dilatih dengan menggunakan Test Set:

In [9]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8682 - loss: 0.3698


[0.3752115070819855, 0.8676000237464905]